In [1]:
from fasttext_comparison_function import eval_dataset

In [7]:
import fasttext
import os
import sys
import os.path
import numpy as np
from numpy import array
from sklearn.model_selection import KFold
import json
import pandas as pd
from sklearn.model_selection import train_test_split
import neattext.functions as nfx
import re

In [85]:
import pandas as pd
# custom data set
data_in_path = r"..\dataset\CLEAN_balanced_data.csv"
data_out_path = r"..\results\output.txt"
df = pd.read_csv(data_in_path)
df = df.dropna()
# df['data'] = "__label__" + df['labels'] + " " + df['descriptions']
# df['data'] = df['labels'] + " " + df['descriptions']
df.head()


,labels,descriptions,repo_name
0,__label__enhancement,"""script path builtin variable (a la `__file__`...",other
1,__label__enhancement,"""give lobby creator extra powers hey",other
2,__label__enhancement,dmtmm congratulations! guys great works past h...,other
3,__label__enhancement,"""router - avoid regex matching possible given ...",other
4,__label__enhancement,"""improve command history currently",other


In [86]:
data_path = r"..\dataset\ft_bal.txt"
# df['data'] = "__label__" + df['labels'] + " " + df['descriptions']
df['data'] = df['labels'] + " " + df['descriptions']
df_file = df[['data']]
df_file.to_csv(data_path,index=False,header=False,sep="\t",encoding='utf-8-sig')
df_file.shape

(29625, 1)

# testing initial model

In [88]:

# path_train = os.path.dirname(__file__) + './tmp/tmp_train.txt'
# path_test = os.path.dirname(__file__) + './tmp/tmp_test.txt'

path_test = r"..\temp\temp_eval.txt"
path_train = r"..\temp\temp_train.txt"
data_out_path = r"..\results\output_ft_regular_TEST.txt"
data_set = data_path
f_out = data_out_path
# try:
print("Converting dataset to array")
# f = open(data_set, 'r+', encoding="UTF-8")

data = array(df_file['data'])
# f.close()

# array for details
fold_outputs = []

# 10 fold loop
kfold = KFold(2, shuffle=True, random_state=1)
fold = 1
for train, test in kfold.split(data):
    print("New tenfold iteration:", str(fold), "-----------------------------------------")
    print("Creating train file")
    tmp_train = data[train]
    pd.DataFrame(tmp_train).to_csv(path_train,index=False,header=False,encoding='utf-8-sig')

    print("Creating test file")
    tmp_test = data[test]
    pd.DataFrame(tmp_test).to_csv(path_test,index=False,header=False,encoding='utf-8-sig')



    print("start training...")
    # use this model if you want to enable auto-tuning. Sadly we did not get it to work during our project
    # model = fasttext.train_supervised(input=path_train, autotuneValidationFile=path_test, autotuneDuration=600)
    # model = fasttext.train_supervised(input=path_train)  # normal fasttext classification without autotuning
    model = fasttext.train_supervised(input=path_train, lr=1.0, epoch=25, wordNgrams=2)
    print("start testing...")
    res = model.test(path_test)
    print(res)
    # get benchmarks
    precision = res[1]
    recall = res[2]
    f1 = 2 * ((precision * recall) / (precision + recall))
    result = {
        '10-Fold iteration:': fold,
        'F1': f1,
        'Recall': recall,
        'Precision': precision
        # 'acuracy'
    }
    # log
    print("Fold over, here are results: ")
    print(json.dumps(result, indent=4))
    fold_outputs.append(result)
    fold += 1
print("Done with 10 fold validation")
# calculate over-all results
mean_recall = 0
mean_precision = 0
mean_f1 = 0
for f in fold_outputs:
    mean_f1 += (f['F1'] / 10)
    mean_recall += (f['Recall'] / 10)
    mean_precision += (f['Precision'] / 10)
    # compile results as json
output = {
    'Results': {
        'F1': mean_f1,
        'Recall': mean_recall,
        'Precision': mean_precision
    },
    'Details': fold_outputs
}
dump = json.dumps(output, indent=4)
print(dump)
# write to output
print("Writing output to file")
o = open(f_out, 'w', encoding="UTF-8")
o.write(dump)
o.close()
# except:
# print("An Error occurred")
# # in any case delete existing temporary files
# finally:
print("Deleting tmp files")
if os.path.exists(path_train):
    os.remove(path_train)
if os.path.exists(path_test):
    os.remove(path_test)
print("Exit.")



Converting dataset to array
New tenfold iteration: 1 -----------------------------------------
Creating train file
Creating test file
start training...
start testing...
(3208, 0.635286783042394, 0.635286783042394)
Fold over, here are results: 
{
    "10-Fold iteration:": 1,
    "F1": 0.635286783042394,
    "Recall": 0.635286783042394,
    "Precision": 0.635286783042394
}
New tenfold iteration: 2 -----------------------------------------
Creating train file
Creating test file
start training...
start testing...
(3198, 0.6125703564727955, 0.6125703564727955)
Fold over, here are results: 
{
    "10-Fold iteration:": 2,
    "F1": 0.6125703564727955,
    "Recall": 0.6125703564727955,
    "Precision": 0.6125703564727955
}
Done with 10 fold validation
{
    "Results": {
        "F1": 0.12478571395151894,
        "Recall": 0.12478571395151894,
        "Precision": 0.12478571395151894
    },
    "Details": [
        {
            "10-Fold iteration:": 1,
            "F1": 0.635286783042394,
    

Plan / work steps
need to split the repo level data into test/train datasets

need to train a model with a simple test/ train split from the balanced data
then use that model to predict on each of the repo datasets

then split each repo dataset into test train

include the repo data inside the training data
then repredict with it
see if it improves it at all. 



Need to organize the data into separate groups for each of the different repo
then run the eval function over each of them. 



# Separating creating model and using it to predict repo issues

In [142]:
import pandas as pd
# custom data set
data_in_path = r"..\dataset\CLEAN_balanced_data.csv"
data_out_path = r"..\results\output.txt"
data_df = pd.read_csv(data_in_path)
data_df = data_df.dropna()
data_df.head()

,labels,descriptions,repo_name
0,__label__enhancement,"""script path builtin variable (a la `__file__`...",other
1,__label__enhancement,"""give lobby creator extra powers hey",other
2,__label__enhancement,dmtmm congratulations! guys great works past h...,other
3,__label__enhancement,"""router - avoid regex matching possible given ...",other
4,__label__enhancement,"""improve command history currently",other


In [143]:
# split dataset to train and dev samples
RANDOM_SEED = 12
x_train, x_test, y_train, y_test = train_test_split(data_df['descriptions'], data_df['labels'],
                                                 test_size=0.3,
                                                 random_state=RANDOM_SEED)
df_train = pd.concat([x_train, y_train], axis=1).reset_index(drop=True)
df_eval = pd.concat([x_test, y_test], axis=1).reset_index(drop=True)

In [144]:
print(df_train.shape)
print(df_eval.shape)
print(data_df.shape)

(20737, 2)
(8888, 2)
(29625, 3)


In [145]:
df_train['data'] = df_train['labels'] + " " + df_train['descriptions']
df_train = df_train[['data']]

df_eval['data'] = df_eval['labels'] + " " + df_eval['descriptions']
df_eval = df_eval[['data']]

In [146]:
print("Creating train file")
df_train.to_csv(path_train,index=False,header=False,encoding='utf-8-sig')

print("Creating test file")
df_eval.to_csv(path_test,index=False,header=False,encoding='utf-8-sig')

Creating train file
Creating test file


In [147]:
model = fasttext.train_supervised(input=path_train, lr=1.0, epoch=25, wordNgrams=2)

In [148]:
# # # save train and eval dataframes to csv
# # get repo data 
df_all = pd.read_csv(r"..\dataset\CLEAN_multiple_projects_benchmark_data.csv")
df_all.head()

,Unnamed: 0,labels,descriptions,repo_name
0,4625,__label__enhancement,clarify sdl_locktexture() description # bug re...,https://api.github.com/repos/libsdl-org/SDL
1,4655,__label__enhancement,[patch] add stadia controller gamecontrollerdb...,https://api.github.com/repos/libsdl-org/SDL
2,4657,__label__enhancement,feature: sdl_openurl() # bug report migrated o...,https://api.github.com/repos/libsdl-org/SDL
3,4730,__label__enhancement,sdl mutex implementation subpar # bug report m...,https://api.github.com/repos/libsdl-org/SDL
4,4750,__label__enhancement,allow desktop opengl contexts egl # bug report...,https://api.github.com/repos/libsdl-org/SDL


In [149]:
df_grp = df_all.groupby(['repo_name']).agg({'descriptions':'count'}).reset_index()
# df_grp.columns = ['Labels','Row Count']
# print(df.head())
df_grp

,repo_name,descriptions
0,https://api.github.com/repos/Kaiserreich/Kaise...,7476
1,https://api.github.com/repos/Microsoft/vscode,8165
2,https://api.github.com/repos/Regalis11/Barotrauma,3769
3,https://api.github.com/repos/dbeaver/dbeaver,6099
4,https://api.github.com/repos/godotengine/godot,13535
5,https://api.github.com/repos/hashicorp/terraform,6816
6,https://api.github.com/repos/libsdl-org/SDL,3379
7,https://api.github.com/repos/microsoft/vscode,8288
8,https://api.github.com/repos/qgis/QGIS,4609
9,https://api.github.com/repos/rstudio/rstudio,2753


In [150]:
repo_names = list(df_all['repo_name'].unique())
repo_names_clean = [repo_name.split('/')[-1] for repo_name in repo_names]
repo_names_clean

['SDL',
 'dbeaver',
 'rstudio',
 'symfony',
 'godot',
 'terraform',
 'QGIS',
 'vscode',
 'vscode',
 'tgstation',
 'Barotrauma',
 'Kaiserreich-4']

In [151]:
path_test = r"..\temp\temp_eval.txt"
path_train = r"..\temp\temp_train.txt"
data_out_path = r"..\results\repo_output_0.3large_train_0.5_repoTrain.json"
f_out = data_out_path
output = []

for i in range(len(repo_names)):
    repo_name_clean= repo_names_clean[i]
    repo_name = repo_names[i]

    repo_df = df_all[df_all['repo_name']==repo_name]
    # split dataset to train and dev samples
    RANDOM_SEED = 12
    x_train, x_test, y_train, y_test = train_test_split(repo_df['descriptions'], repo_df['labels'],
                                                 test_size=0.5,
                                                 random_state=RANDOM_SEED)
    df_train_r = pd.concat([x_train, y_train], axis=1).reset_index(drop=True)
    df_eval_r = pd.concat([x_test, y_test], axis=1).reset_index(drop=True)

    df_train_r['data'] = df_train_r['labels'] + " " + df_train_r['descriptions']
    df_train_r = df_train_r[['data']]

    df_eval_r['data'] = df_eval_r['labels'] + " " + df_eval_r['descriptions']
    df_eval_r = df_eval_r[['data']]



    print("Creating test file")
    df_eval_r.to_csv(path_test,index=False,header=False,encoding='utf-8-sig')

    # now perform an eval with the eval dataset,
    
    # then combine the train repo with the overall training dataset 
    # now evaluate the performance of the larger trained dataset

    print("start testing...")
    print(f"test {repo_name_clean}")
    res = model.test(path_test)
    print(res)
    # get benchmarks
    precision = res[1]
    recall = res[2]
    f1 = 2 * ((precision * recall) / (precision + recall))
    result_balanced = {
        'F1': f1,
        'Recall': recall,
        'Precision': precision
    }

    print("testing over")
    # print(json.dumps(result_balanced, indent=4))

    df_train_repo_all = df_train.append(df_train_r)
    df_train_repo_all.to_csv(path_train,index=False,header=False,encoding='utf-8-sig')

    model_2 = fasttext.train_supervised(input=path_train, lr=1.0, epoch=25, wordNgrams=2)
    res = model_2.test(path_test)
    print(f"augment {repo_name_clean}")
    print(res)

    precision = res[1]
    recall = res[2]
    f1 = 2 * ((precision * recall) / (precision + recall))

    result_augmented = {
        'F1': f1,
        'Recall': recall,
        'Precision': precision
    }
    print("Augment over")
    # print(json.dumps(result_augmented, indent=4))

    output_temp = {
        'RESULTS FOR': repo_name_clean,
        'result_balanced': result_balanced,
        'result_augmented': result_augmented
    }

    output.append(output_temp)

dump = json.dumps(output, indent=4)
print(dump)
# write to output
print("Writing output to file")
# f_out = data_out_path + f"\REPO_output.txt"
o = open(f_out, 'w', encoding="UTF-8")
o.write(dump)
o.close()
# except:
# print("An Error occurred")
# # in any case delete existing temporary files
# finally:
print("Deleting tmp files")
if os.path.exists(path_train):
    os.remove(path_train)
if os.path.exists(path_test):
    os.remove(path_test)
print("Exit.")

    

Creating test file
start testing...
test SDL
(1, 1.0, 1.0)
testing over
augment SDL
(1, 1.0, 1.0)
Augment over
Creating test file
start testing...
test dbeaver
(415, 0.4674698795180723, 0.4674698795180723)
testing over
augment dbeaver
(415, 0.653012048192771, 0.653012048192771)
Augment over
Creating test file
start testing...
test rstudio
(79, 0.6075949367088608, 0.6075949367088608)
testing over
augment rstudio
(79, 0.7215189873417721, 0.7215189873417721)
Augment over
Creating test file
start testing...
test symfony
(148, 0.5, 0.5)
testing over
augment symfony
(148, 0.9121621621621622, 0.9121621621621622)
Augment over
Creating test file
start testing...
test godot
(633, 0.6540284360189573, 0.6540284360189573)
testing over
augment godot
(633, 0.8688783570300158, 0.8688783570300158)
Augment over
Creating test file
start testing...
test terraform
(164, 0.7073170731707317, 0.7073170731707317)
testing over
augment terraform
(164, 0.7926829268292683, 0.7926829268292683)
Augment over
Creating